In [23]:
%load_ext autoreload
%autoreload 2

from hydroml.training.train import train
from hydroml.config.config import Config, load_config
import hydroml.training.finetune as ft
from hydroml.models import get_model_from_path
from hydroml.data.camels_aus_ds import get_dataset
from hydroml.prediction.prediction import process_and_convert_dataloader_to_xarray
from hydroml.models.get_model_from_path import get_model_from_path
from hydroml.evaluation.metrics import get_metrics
from hydroml.training.trainer import get_trainer



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
with open('P://work//sho108//hydroml//workflows//basins.txt', 'r') as f:
    basins = f.read().splitlines()

# '410730'
config = Config(cal={'periods' : [['1985-01-01', '2014-01-01']], 'catchment_ids':['410730']}, #, 'weights': [1, 1, 1, 1, 1, 1, 1, 1, 1]
                val={'periods' : [['1985-01-01', '2014-01-01']], 'catchment_ids':['410730']}, #,
                max_epochs=10,
                lstm_hidden_size=64,
                batch_size=128,
                initial_forget_bias=0,
                parent_path = 'P://work//sho108//hydroml//results_2',
                lr=1e-5
                
                )

dataset_val_train = get_dataset(config, 'val', is_train=True)
dataloader_val_train = dataset_val_train.to_dataloader()

dataset_cal_train = get_dataset(config, 'cal', is_train=True)
dataloader_cal_train = dataset_cal_train.to_dataloader()

dataset_val_test = get_dataset(config, 'val', is_train=False)
dataloader_val_test = dataset_val_test.to_dataloader()

dataset_cal_test = get_dataset(config, 'cal', is_train=False)
dataloader_cal_test = dataset_cal_test.to_dataloader()


Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml


Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml


Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml


Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml


In [30]:
current_path, version = train(config)

Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml
valid data points per catchment {0: 10441}


GPU available: False, used: False                                      
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Transforming data: loading transform parameters from P:\work\sho108\hydroml\results_2\default\params.yaml


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory \\fs1-cbr.nexus.csiro.au\{ev-ca-macq}\work\sho108\hydroml\results_2\default\241210133745_d65e exists and is not empty.

  | Name              | Type       | Params | Mode 
---------------------------------------------------------
0 | static_embedding  | Linear     | 15     | train
1 | dynamic_embedding | Linear     | 5      | train
2 | lstm              | LSTM       | 17.4 K | train
3 | dropout           | Identity   | 0      | train
4 | head              | Sequential | 661    | train
---------------------------------------------------------
18.1 K    Trainable params
0         Non-trainable params
18.1 K    Total params
0.072     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [31]:
print('first pass:')
model = get_model_from_path(current_path/version)

ds = process_and_convert_dataloader_to_xarray(dataloader_cal_train, model, config)
m = get_metrics(ds)
m.to_dataframe()

first pass:


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\xarray\core\computation.py:818: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


,,,nse,kge,rmse,bias,relative_bias,absolute_bias,nse_sqrt
catchment_id,lead_time,feature,,,,,,,
410730,0,0,0.257901,0.239558,0.96454,0.961069,-0.038931,0.961069,0.372921


In [36]:
import pandas as pd
import numpy as np
import xarray as xr
import tensorboard
pd.__version__, np.__version__, xr.__version__, tensorboard.__version__


('2.2.3', '1.26.4', '2024.11.0', '2.18.0')

In [32]:
print('second pass:')
trainer = get_trainer(config)
trainer.fit(model, dataloader_cal_train)


model = get_model_from_path(current_path/version, check_point='last-v1')

ds = process_and_convert_dataloader_to_xarray(dataloader_cal_train, model, config)
m = get_metrics(ds)
m.to_dataframe()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


second pass:


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory \\fs1-cbr.nexus.csiro.au\{ev-ca-macq}\work\sho108\hydroml\results_2\default\241210133745_d65e exists and is not empty.

  | Name              | Type       | Params | Mode 
---------------------------------------------------------
0 | static_embedding  | Linear     | 15     | train
1 | dynamic_embedding | Linear     | 5      | train
2 | lstm              | LSTM       | 17.4 K | train
3 | dropout           | Identity   | 0      | train
4 | head              | Sequential | 661    | train
---------------------------------------------------------
18.1 K    Trainable params
0         Non-t

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\xarray\core\computation.py:818: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


,,,nse,kge,rmse,bias,relative_bias,absolute_bias,nse_sqrt
catchment_id,lead_time,feature,,,,,,,
410730,0,0,0.259863,0.270233,0.963264,0.905444,-0.094556,0.905444,0.365115


In [33]:
print('third pass:')
trainer = get_trainer(config)
trainer.fit(model, dataloader_cal_train)


model = get_model_from_path(current_path/version, check_point='last-v2')

ds = process_and_convert_dataloader_to_xarray(dataloader_cal_train, model, config)
m = get_metrics(ds)
m.to_dataframe()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory \\fs1-cbr.nexus.csiro.au\{ev-ca-macq}\work\sho108\hydroml\results_2\default\241210133745_d65e exists and is not empty.

  | Name              | Type       | Params | Mode 
---------------------------------------------------------
0 | static_embedding  | Linear     | 15     | train
1 | dynamic_embedding | Linear     | 5      | train
2 | lstm              | LSTM       | 17.4 K | train
3 | dropout           | Identity   | 0      | train
4 | head              | Sequential | 661 

third pass:


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\xarray\core\computation.py:818: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


,,,nse,kge,rmse,bias,relative_bias,absolute_bias,nse_sqrt
catchment_id,lead_time,feature,,,,,,,
410730,0,0,0.339572,0.312635,0.909918,0.803561,-0.196439,0.803561,0.446969
